This notebook run Google Colab(A100 GPU).

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "cyberagent/Mistral-Nemo-Japanese-Instruct-2408"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
def generate_new_corpus(original_corpus, original_likability, original_mood, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for changed_likability in [1, 3, 5]:
        for changed_mood in [1, 3, 5]:
            if changed_likability == original_likability and changed_mood == original_mood:
                continue

            prompt = f"""あなたのタスクは「既存の日常会話コーパスを指示に従って変更し、新しいコーパスを生成すること」です。
以下の指示に基づいて、自然で一貫性のある新しいコーパスを生成してください：

## 指示
変更前のコーパスとその設定、及び変更後のコーパスの設定が与えられます。
変更後のコーパスの設定に基づいて、変更前のコーパスを適切に変更してください。
主に変更するのはBの会話内容についてで、Aの口調や会話テーマについては大きく変更しないでください。
使用できる絵文字は次に挙げるものだけです。それ以外は使用しないでください。
😊,🥰,😉,🤗,😭,🤣,😆,🙄,😡,😲,😳,😔,😇,😙,🥳,🤔,🥺,🥱,💓,✨

設定に基づく会話の変化については、以下を参考にしてください。
- Aに対するBの好感度(1～5):
    - 1の例(初対面の人と話すような口調、敬語)
        A: こんにちは。何してるんですか？
        B: こんにちは。今は特に何もしていませんでした。
    - 3の例(仲の良い友達や知り合いと話すような口調、くだけた表現)
        A: こんにちは。何してるの？
        B: こんにちは！特に何もしてないよ？
    - 5の例(恋人と話すような口調、くだけた表現)
        A: 今何してんの？
        B: 何もしてないー。なんかしようよー。

- Bの機嫌(1～5):
    - 1の例(とても機嫌が悪い、イライラしている、ネガティブな発言、絵文字や感嘆符は使用しない)
        A: 最近面白かったことありますか？
        B: ない。そんなこと聞かないで。
    - 3の例(機嫌は良くも悪くもない、通常の会話、絵文字や感嘆符を適度に使用する)
        A: 最近面白かったことありますか？
        B: うーん、ないかも。あなたはどう？
    - 5の例(とても機嫌が良い、ポジティブな発言、絵文字や感嘆符を多用する)
        A: 最近面白かったことありますか？
        B: えー、ないなー😔逆に聞かせてよ！どんなことがあったのー？

### 変更前のコーパス
{original_corpus}

### 変更前のコーパスの設定
- Aに対するBの好感度: {original_likability}
- Bの機嫌: {original_mood}

### 変更後のコーパスの設定
- Aに対するBの好感度: {changed_likability}
- Bの機嫌: {changed_mood}

### 出力形式
{{変更の概要(例: 変更前のコーパスについて、Bの会話内容を「初対面の人と話す口調」で「とても機嫌が悪い」のように変更する。)}}

{{変更後のコーパス}}
"""

            messages = [
                {"role": "system", "content": "あなたは親切なAIアシスタントです。"},
                {"role": "user", "content": prompt}
            ]

            input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
            output_ids = model.generate(input_ids,
                                        max_new_tokens=1024,
                                        do_sample=True,
                                        top_k=50,
                                        top_p=0.9,
                                        num_return_sequences=5
                                        )

            for i, output_id in enumerate(output_ids):
                text = tokenizer.decode(output_id[input_ids.shape[-1]:], skip_special_tokens=True)

                with open(f"{output_dir}/corpus_{changed_likability}_{changed_mood}_{i}.txt", "w") as f:
                    f.write(text)

In [28]:
import json

def convert_conversation_with_attributes_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    conversations = json_data["conversations"]

    result = []

    likability = None
    mood = None

    for conversation in conversations:
        if conversation["role"] == "user":
            role = "A"
        elif conversation["role"] == "assistant":
            role = "B"

            if "attribute" in conversation:
                likability = conversation["attribute"].get("likability", None)
                mood = conversation["attribute"].get("mood", None)

        result.append(f"{role}: {conversation['content']}")

    conversation_text = "\n".join(result)

    return (conversation_text, likability, mood)

def txt_to_json(file_path):
    file_name = file_path.split('/')[-1]
    _, likability, mood, _ = file_name.split('_')

    likability = int(likability)
    mood = int(mood)

    with open(file_path, 'r', encoding='utf-8') as file:
        txt = file.read()

    lines = txt.splitlines()

    conversation = []
    conversation_started = False
    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith('A:') or line.startswith('B:'):
            conversation_started = True

        if conversation_started:
            if line.startswith('A:'):
                content = line[2:].strip()
                conversation.append({
                    "role": "user",
                    "content": content
                })
            elif line.startswith('B:'):
                content = line[2:].strip()
                conversation.append({
                    "role": "assistant",
                    "content": content,
                    "attribute": {
                        "likability": likability,
                        "mood": mood
                    }
                })
            else:
                break

    output_json = {
        "conversations": conversation
    }

    dump_json_data = json.dumps(output_json, ensure_ascii=False, indent=2)

    with open(f"{file_path.split('.')[0]}.json", 'w', encoding='utf-8') as file:
        file.write(dump_json_data)

In [ ]:
import os
import re

from tqdm import tqdm

conversations_dir = "/content/drive/MyDrive/AKU/dataset"
file_list = [os.path.join(conversations_dir, file) for file in os.listdir(conversations_dir) if file.endswith(".json")]

def sort_key(file_name):
    number = re.findall(r'\d+', file_name)
    return int(number[0]) if number else 0

sorted_list = sorted(file_list, key=sort_key)

for conv_path in tqdm(sorted_list):
    output_dir = conv_path.split('.')[0]+"_gen"

    original_corpus, original_likability, original_mood = convert_conversation_with_attributes_from_file(conv_path)
    generate_new_corpus(original_corpus, original_likability, original_mood, output_dir)

    for file in os.listdir(output_dir):
        if file.endswith(".txt"):
            txt_to_json(os.path.join(output_dir, file))
            os.remove(os.path.join(output_dir, file))